In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Install required packages\n",
    "!pip install --upgrade transformers torch peft huggingface-hub\n",
    "\n",
    "# Log in to Hugging Face (run this cell and paste your token when prompted)\n",
    "!huggingface-cli login\n",
    "\n",
    "# Import libraries\n",
    "import os\n",
    "import torch\n",
    "from transformers import AutoTokenizer, AutoModelForCausalLM\n",
    "from peft import PeftModel, LoraConfig\n",
    "\n",
    "# Define cache directory\n",
    "cache_dir = \"./pretrained-models\"\n",
    "os.makedirs(cache_dir, exist_ok=True)\n",
    "\n",
    "# Load Llama-3.1-8B base model\n",
    "llama3_base = AutoModelForCausalLM.from_pretrained(\n",
    "    \"meta-llama/Llama-3.1-8B\",\n",
    "    trust_remote_code=True,\n",
    "    device_map=\"auto\",\n",
    "    cache_dir=cache_dir,\n",
    "    torch_dtype=torch.float16,\n",
    ")\n",
    "\n",
    "# Load tokenizer\n",
    "llama3_tokenizer = AutoTokenizer.from_pretrained(\"meta-llama/Llama-3.1-8B\", cache_dir=cache_dir)\n",
    "\n",
    "# LoRA Fine-Tuning for Llama-3.1-8B\n",
    "lora_config = LoraConfig(\n",
    "    r=8,\n",
    "    lora_alpha=16,\n",
    "    lora_dropout=0.1,\n",
    "    target_modules=[\"q_proj\", \"v_proj\"],\n",
    "    task_type=\"CAUSAL_LM\",\n",
    ")\n",
    "\n",
    "llama3_model = get_peft_model(llama3_base, lora_config)\n",
    "llama3_model.load_adapter(\n",
    "    \"./finetuned_models/dow30-202305-202405-llama-3.1-8b_202602280001\",  # Placeholder path\n",
    "    cache_dir=cache_dir,\n",
    ")\n",
    "llama3_model = llama3_model.eval()\n",
    "\n",
    "# Testing function for Llama-3.1-8B\n",
    "def test_demo(model, tokenizer, prompt):\n",
    "    inputs = tokenizer(prompt, return_tensors=\"pt\", padding=False, max_length=8000)\n",
    "    inputs = {key: value.to(model.device) for key, value in inputs.items()}\n",
    "    \n",
    "    start_time = time.time()\n",
    "    outputs = model.generate(\n",
    "        **inputs,\n",
    "        max_length=4096,\n",
    "        do_sample=True,\n",
    "        eos_token_id=tokenizer.eos_token_id,\n",
    "        use_cache=True,\n",
    "    )\n",
    "    end_time = time.time()\n",
    "    \n",
    "    output = tokenizer.decode(outputs[0], skip_special_tokens=True)\n",
    "    return output, end_time - start_time\n",
    "\n",
    "# Example usage (replace with your dataset)\n",
    "prompt = \"You are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. Ticker: AXP\"\n",
    "output, response_time = test_demo(llama3_model, llama3_tokenizer, prompt)\n",
    "print(f\"Prediction: {output}\")\n",
    "print(f\"Response Time: {response_time:.4f} seconds\")\n"
   ]
  }
 ],
 "metadata": {
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
}